In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_pickle('Cleaned_ed.pkl')

# MODELING

In [4]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from keras.layers import Dense

/Users/leticiabeeck/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/leticiabeeck/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
binary = {'Meeting Target':1,'Exceeding Target':1,
           'Not Meeting Target':0,'Approaching Target':0}
df['Student Achievement Rating'] = df['Student Achievement Rating'].map(binary)

In [6]:
df['Student Achievement Rating'].value_counts()

1    589
0    206
Name: Student Achievement Rating, dtype: int64

## Train Test Split 

In [7]:
df['City'] = [value if value in df['City'].value_counts().index[:4] else 'Other' for value in df['City']]

In [10]:
cats= ['City', 'Grades',
       'Rigorous Instruction Rating', 'Collaborative Teachers Rating',
       'Supportive Environment Rating', 'Effective School Leadership Rating',
       'Strong Family-Community Ties Rating', 'Trust Rating']
df.drop(['School Name', 'Location Code', 'Address (Full)'], axis=1, inplace=True)

In [11]:
X = df.drop('Student Achievement Rating',axis=1)
y = df['Student Achievement Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


X_train = pd.get_dummies(X_train, columns=cats, drop_first=True)
X_test = pd.get_dummies(X_test, columns=cats, drop_first=True)

In [12]:
[col for col in X_train.columns if col not in X_test.columns]

['Grades_02,03,04,05',
 'Grades_03,04,05',
 'Grades_03,04,05,06',
 'Grades_04,05',
 'Grades_04,05,06,07,08',
 'Grades_06,07,08',
 'Grades_0K,01,02,03,04,05,06,07',
 'Grades_0K,01,02,03,04,05,SE',
 'Grades_PK,0K,01,02',
 'Grades_PK,0K,01,02,03',
 'Grades_PK,0K,01,02,03,04',
 'Grades_PK,0K,01,02,03,04,05,06,07,08,SE',
 'Grades_PK,0K,01,02,03,04,05,07,08']

In [13]:
df['has_preschool'] = df['Grades'].str.contains('PK','0K')
df['has_middle_school']=df['Grades'].str.contains('05','06')
df['has_junior_high']=df['Grades'].str.contains('07','08')

In [14]:
df['Grades'].str.contains('05','06').mean()

0.9823899371069182

In [15]:
df['Grades'].unique()

array(['PK,0K,01,02,03,04,05', 'PK,0K,01,02,03,04,05,06,07,08',
       '0K,01,02,03,04,05,06,07,08,09,10,11,12', '0K,01,02,03,04,05',
       '0K,01,02,03,04,05,06,07,08', '04,05,06,07,08', '03,04,05',
       '0K,01,02,03,04,05,06,07,08,SE', 'PK,0K,01,02,03,04,05,06',
       'PK,0K,01,02,03,04,05,SE', 'PK,0K,01,02,03', '04,05',
       '0K,01,02,03,04', '0K,01,02,03,04,05,06', 'PK,0K,01,02',
       'PK,0K,01,02,03,04', '02,03,04,05', 'PK,0K,01,02,03,04,05,07,08',
       'PK,0K,01,02,03,04,05,06,07,08,SE', '06,07,08', '0K',
       '03,04,05,06', '0K,01,02,03,04,05,SE', '01,05,06,07,08,K',
       '01,02,03,04,05,06,07,08', '0K,01,02,03,04,05,06,07'], dtype=object)

# TPOT Classifier 

In [31]:
from tpot import TPOTClassifier

In [43]:
pipeline_optimizer = TPOTClassifier(generations=1, population_size=20,cv=5, verbosity=2,
                      config_dict='TPOT light')

In [45]:
pipeline_optimizer.fit(X_train,y_train)

Generation 1 - Current best internal CV score: 0.7957019044494438

Best pipeline: DecisionTreeClassifier(Normalizer(input_matrix, norm=l1), criterion=gini, max_depth=7, min_samples_leaf=20, min_samples_split=3)


TPOTClassifier(config_dict={'sklearn.naive_bayes.GaussianNB': {}, 'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.tree.DecisionT...e_selection.VarianceThreshold': {'threshold': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2]}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=1, max_eval_time_mins=5,
        max_time_mins=None, memory=None, mutation_rate=0.9, n_jobs=1,
        offspring_size=20, periodic_checkpoint_folder=None,
        population_size=20, random_state=None, scoring=None, subsample=1.0,
        verbosity=2, warm_start=False)